In [1]:
import pandas as pd
from itertools import combinations
from tqdm import tqdm
import numpy as np

In [264]:
def data_prep_lift_fix(data_for_lift):
    columns_to_combine = ['Sex', 'Smoke Status', 'Chromosome', 'Hugo_Symbol', 'SNP_event',
                          "Consequence", 'Exon_Number', "Diagnosis Age", "TMB (nonsynonymous)",
                          "Position", "Protein_position", "Codons", "VAR_TYPE_SX"]

    # Calculate cancer probabilities using the correct dataframe
    cancer_probabilities = {
        cancer_type: data_for_lift[cancer_type].mean()
        for cancer_type in data_for_lift.select_dtypes(include=['bool', 'int']).columns
    }

    # Create feature combinations
    feature_combinations = list(combinations(columns_to_combine, 5))

    return cancer_probabilities, feature_combinations

In [199]:
def data_prep_lift(df):
    # Select a subset of columns to analyze (e.g., most relevant ones)
    columns_to_combine = ['Sex', 'Smoke Status', 'Chromosome', 'Hugo_Symbol', 'SNP_event', "Consequence", 'Exon_Number', "Diagnosis Age", "TMB (nonsynonymous)", "Position", "Protein_position", "Codons", "VAR_TYPE_SX"]

    cancer_probabilities = {cancer_type: data_for_lift[cancer_type].mean() for cancer_type in list(df["Cancer Type"].unique())}
    feature_combinations = list(combinations(columns_to_combine, 5))

    return cancer_probabilities, feature_combinations

In [296]:
P_B = cancer_probabilities['Gallbladder Carcinoma']
data_for_lift = pd.read_csv("./pan_cancer/data_for_lift.csv", index_col=0)

FileNotFoundError: [Errno 2] No such file or directory: './pan_cancer/data_for_lift.csv'

In [301]:
lifts = []
feature = ('Smoke Status', 'Consequence', 'Diagnosis Age', 'TMB (nonsynonymous)', 'VAR_TYPE_SX')
cancer_type = 'Gallbladder Carcinoma'

# Combine the selected features into a single feature
combined_feature = data_for_lift[list(feature)].astype(str).agg('_'.join, axis=1)
combined_feature.reset_index().drop_duplicates().set_index("PATIENT_ID", inplace=True)

# Compute value counts for the combined feature
combined_counts = combined_feature.value_counts()
valid_features = combined_counts[combined_counts >= 50].index

# Filter the combined feature to include only valid entries
filtered_data = combined_feature[combined_feature.isin(valid_features)]

# Reset index to align with the original DataFrame for filtering the cancer type
filtered_data = filtered_data.reset_index(drop=True)
cancer_data = data_for_lift[cancer_type].reset_index(drop=True)  # Make sure cancer_data has the same index

# Compute joint probabilities for cancer type
joint_prob = (
    filtered_data[cancer_data == 1]
    .value_counts(normalize=True)
    .reindex(filtered_data.value_counts(normalize=True).index, fill_value=0)
)

# Calculate lift
P_A = filtered_data.value_counts(normalize=True)
lift = (joint_prob / (P_A * P_B)).round(2)

In [443]:
lifts = []
feature = ('Smoke Status', 'Consequence', 'Diagnosis Age', 'TMB (nonsynonymous)', 'VAR_TYPE_SX')
cancer_type = 'Gallbladder Carcinoma'

# Count the number of unique PATIENT_IDs per feature combination
feature_counts = (
    data_for_lift.reset_index()
    .groupby(list(feature))['PATIENT_ID']
    .nunique()
    .reset_index(name='patient_count')
)

# Create the feature combination column
feature_counts["feature_combination"] = feature_counts[list(feature)].fillna('missing').astype(str).agg('_'.join, axis=1)

feature_counts = feature_counts[feature_counts["patient_count"] >= 50]
# Filter valid feature combinations with at least 50 unique patients
valid_features = set(feature_counts["feature_combination"].astype(str).unique())

# Combine features in the original dataset
combined_feature = data_for_lift[list(feature)].fillna('missing').astype(str).agg('_'.join, axis=1)

# Apply mask for valid features
valid_mask = combined_feature.astype(str).isin(valid_features)
filtered_data = combined_feature[valid_mask]
cancer_data = data_for_lift.loc[valid_mask, cancer_type]

# Compute probabilities
P_A = filtered_data.value_counts(normalize=True)

# Calculate joint probability more efficiently
joint_counts = pd.Series(0, index=P_A.index)
positive_counts = filtered_data[cancer_data == 1].value_counts()
joint_counts.update(positive_counts)
joint_prob = joint_counts / len(filtered_data)

# Calculate lift
lift = (joint_prob / (P_A * P_B)).round(2)


In [448]:
lift#[lift.index == "Unknown_missense_variant_51-60_3.233333333_Substitution/Indel"]
# Create a valid_mask ensuring both sets match correctly
# combined_feature.name ==


Unknown_missense_variant_61-70_0.266666667_Substitution/Indel    2.33
Unknown_missense_variant_51-60_0.3_Substitution/Indel            3.46
Unknown_missense_variant_51-60_0.233333333_Substitution/Indel    1.56
Unknown_missense_variant_61-70_0.233333333_Substitution/Indel    2.49
Unknown_missense_variant_51-60_0.2_Substitution/Indel            1.18
                                                                 ... 
Unknown_stop_gained_41-50_0.166666667_Truncation                 0.65
Unknown_stop_gained_51-60_0.1_Truncation                         0.68
Unknown_frameshift_variant_51-60_0.1_Truncation                  3.41
Unknown_missense_variant_51-60_0.033333333_Substitution/Indel    0.74
Unknown_missense_variant_41-50_0.033333333_Substitution/Indel    1.68
Length: 82, dtype: float64

In [ ]:
if __name__ == "__main__":
    data_for_lift = pd.read_csv("./pan_cancer/data_for_lift.csv", index_col=0)
    cancer_prob, features_comb = data_prep_lift(data_for_lift)
    lifts_df = calculate_lift(data_for_lift, cancer_prob, features_comb)
    # lifts_df.to_csv("lifts.csv", index=False)

In [310]:
combined_feature[combined_feature.index == "Unknown_missense_variant_51-60_3.233333333_Substitution/Indel"]
# combined_counts
# combined_feature[combined_feature.index == "Female_Unknown_71-80_6.166666667_Substitution/Indel"]
# combined_feature[combined_feature.index == "Patient8178"]
# joint_prob[joint_prob.index == "Female_Unknown_71-80_6.166666667_Substitution/Indel"]
# P_A[P_A.index == "Female_Unknown_71-80_6.166666667_Substitution/Indel"]
# joint_prob / (P_A * P_B)

Series([], dtype: object)

In [292]:
cancer_probabilities, features_combinations = data_prep_lift_fix(data_for_lift)

In [13]:
lift_df = pd.read_csv("lifts_new.csv", index_col=0)
data_for_lift = pd.read_csv("data_for_lift.csv", index_col=0)

In [17]:
# try_df = data_for_lift[data_for_lift["Cancer Type"] == "Colorectal Carcinoma"]
try_df = data_for_lift[data_for_lift['Smoke Status'] == "Unknown"]
try_df = try_df[try_df['SNP_event'] == "C>G"]
try_df = try_df[try_df['Exon_Number'] == "02/05"]
try_df = try_df[try_df['Protein_position'] == 12.0]
try_df = try_df[try_df['Codons'] == "Ggt/Cgt"]

# set(try_df.index)
try_df

,Cancer Type,Cancer Type Detailed,Tumor Stage,Sample Type,Sex,Diagnosis Age,Smoke Status,TMB (nonsynonymous),Hugo_Symbol,Chromosome,...,Extrahepatic Cholangiocarcinoma,Gallbladder Carcinoma,Gastric Cancer,Intrahepatic Cholangiocarcinoma,Liver Hepatocellular Carcinoma,Non Small Cell Lung Cancer,Pancreatic Cancer,Small Cell Lung Cancer,Soft Tissue Sarcoma,Position
PATIENT_ID,,,,,,,,,,,,,,,,,,,,,
Patient0257,Pancreatic Cancer,Pancreatic Adenocarcinoma,IV,Metastasis,Male,61-70,Unknown,0.133333,KRAS,12,...,0,0,0,0,0,0,1,0,0,25398285.0-25398285.0
Patient0295,Non Small Cell Lung Cancer,Large Cell Lung Carcinoma,IV,Metastasis,Male,51-60,Unknown,1.866667,KRAS,12,...,0,0,0,0,0,1,0,0,0,25398285.0-25398285.0
Patient0378,Intrahepatic Cholangiocarcinoma,Intrahepatic Cholangiocarcinoma,Unknown,Primary,Male,51-60,Unknown,0.233333,KRAS,12,...,0,0,0,1,0,0,0,0,0,25398285.0-25398285.0
Patient0557,Pancreatic Cancer,Pancreatic Adenocarcinoma,IV,Metastasis,Male,71-80,Unknown,0.166667,KRAS,12,...,0,0,0,0,0,0,1,0,0,25398285.0-25398285.0
Patient0582,Breast Carcinoma,Breast Invasive Carcinoma,IV,Metastasis,Female,41-50,Unknown,1.200000,KRAS,12,...,0,0,0,0,0,0,0,0,0,25398285.0-25398285.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Patient8524,Pancreatic Cancer,Pancreatic Adenocarcinoma,II,Primary,Male,51-60,Unknown,0.166667,KRAS,12,...,0,0,0,0,0,0,1,0,0,25398285.0-25398285.0
Patient8538,Pancreatic Cancer,Pancreatic Adenocarcinoma,I,Primary,Male,41-50,Unknown,0.166667,KRAS,12,...,0,0,0,0,0,0,1,0,0,25398285.0-25398285.0
Patient8546,Pancreatic Cancer,Pancreatic Adenocarcinoma,II,Primary,Female,61-70,Unknown,0.100000,KRAS,12,...,0,0,0,0,0,0,1,0,0,25398285.0-25398285.0


In [11]:
lift_df.sort_values(by="Lift Value", ascending=False).head(1000)

,Feature Combination,Feature,Lift Value
Cancer Type,,,
Pancreatic Cancer,"('Smoke Status', 'Chromosome', 'Hugo_Symbol', ...",Unknown_12_KRAS_missense_variant_Ggt/Cgt,27.03
Pancreatic Cancer,"('Smoke Status', 'Chromosome', 'Hugo_Symbol', ...",Unknown_12_KRAS_C>G_Ggt/Cgt,27.03
Pancreatic Cancer,"('Smoke Status', 'Hugo_Symbol', 'SNP_event', '...",Unknown_KRAS_C>G_25398285.0-25398285.0_Substit...,27.03
Pancreatic Cancer,"('Smoke Status', 'Hugo_Symbol', 'SNP_event', '...",Unknown_KRAS_C>G_12.0_Ggt/Cgt,27.03
Pancreatic Cancer,"('Smoke Status', 'Hugo_Symbol', 'Consequence',...",Unknown_KRAS_missense_variant_02/05_Ggt/Cgt,27.03
...,...,...,...
Pancreatic Cancer,"('Hugo_Symbol', 'Consequence', 'Exon_Number', ...",KRAS_missense_variant_02/05_0.133333333_12.0,16.64
Pancreatic Cancer,"('Smoke Status', 'Exon_Number', 'Diagnosis Age...",Unknown_02/05_51-60_25398284.0-25398284.0_gGt/gTt,16.64
Pancreatic Cancer,"('Smoke Status', 'SNP_event', 'Diagnosis Age',...",Unknown_C>A_51-60_25398284.0-25398284.0_gGt/gTt,16.64


In [19]:
df_combined = lift_df.groupby(["Cancer Type", "Lift Value"]).apply(
    lambda x: dict(zip(x["Feature Combination"], x["Feature"]))
).reset_index(name="Feature-Value Pairs")

C:\Users\talho\AppData\Local\Temp\ipykernel_8600\589612458.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_combined = lift_df.groupby(["Cancer Type", "Lift Value"]).apply(


In [22]:
df = lift_df.copy()

In [35]:
# Convert feature sets from strings to actual sets for easier comparison
df["Feature Set"] = df["Feature Combination"].apply(eval)

# Group by Cancer Type, Lift, and Feature Values
grouped = df.groupby(["Cancer Type", "Lift Value", "Feature"])

# Function to merge feature sets that share identical feature values
def merge_identical_feature_values(group):
    merged_sets = []

    for feature_set in group["Feature Set"]:
        merged = False
        for i, existing_set in enumerate(merged_sets):
            if feature_set & existing_set:  # Overlapping features
                merged_sets[i] = existing_set | feature_set  # Merge sets
                merged = True
                break
        if not merged:
            merged_sets.append(feature_set)

    return pd.DataFrame({
        "Merged Features": [str(fs) for fs in merged_sets],
        "Feature": group["Feature"].iloc[0]  # Keep one unique value
    })

# Apply merging function to each group
merged_df = grouped.apply(merge_identical_feature_values).reset_index()

# Drop unnecessary index column
merged_df = merged_df.drop(columns=["level_3"])

C:\Users\talho\AppData\Local\Temp\ipykernel_8600\214869001.py:27: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  merged_df = grouped.apply(merge_identical_feature_values).reset_index()


ValueError: cannot insert Feature, already exists

In [29]:
grouped.sort_values(by="Lift Value", ascending=False).head(1000)


,Cancer Type,Lift Value,Merged Features,Merged Values
6529,Pancreatic Cancer,27.03,"[[Chromosome, Codons, Consequence, Exon_Number...",[[Unknown_02/05_12.0_Ggt/Cgt_Substitution/Inde...
6528,Pancreatic Cancer,26.69,"[[Chromosome, Codons, Consequence, Exon_Number...",[[02/05_25398285.0-25398285.0_12.0_Ggt/Cgt_Sub...
6527,Pancreatic Cancer,26.36,"[[Codons, Consequence, Protein_position, SNP_e...",[[C>G_missense_variant_12.0_Ggt/Cgt_Substituti...
999,Breast Carcinoma,26.22,"[[Chromosome, Codons, Consequence, Exon_Number...",[[Female_Unknown_1047.0_cAt/cGt_Substitution/I...
6526,Pancreatic Cancer,26.03,"[[Chromosome, Codons, Consequence, SNP_event, ...",[[12_C>G_missense_variant_Ggt/Cgt_Substitution...
...,...,...,...,...
710,Breast Carcinoma,7.86,"[[Chromosome, Codons, Diagnosis Age, SNP_event...",[[Female_Unknown_3_0.233333333_Substitution/In...
5960,Pancreatic Cancer,7.86,"[[Consequence, Diagnosis Age, Exon_Number, SNP...",[[C>T_missense_variant_02/05_41-50_Substitutio...
5959,Pancreatic Cancer,7.85,"[[Codons, Consequence, Diagnosis Age, Protein_...",[[C>A_missense_variant_61-70_12.0_Substitution...
4803,Liver Hepatocellular Carcinoma,7.85,"[[Chromosome, Consequence, Hugo_Symbol, Smoke ...",[[Unknown_16_AXIN1_stop_gained_Truncation]]


In [40]:
df = lift_df.copy()

In [41]:
def parse_features(row):
    feature_names = row["Feature Combination"].split("_")
    feature_values = row["Feature"].split("_")
    return dict(zip(feature_names, feature_values))

df["Feature Dict"] = df.apply(parse_features, axis=1)

In [ ]:
# Function to find matching patient IDs
def find_matching_patients(row, data_for_lift):
    matching_ids = []

    for _, patient_row in data_for_lift.iterrows():
        match = all(patient_row.get(feature, None) == value for feature, value in row["Feature Dict"].items())
        if match:
            matching_ids.append(patient_row["PATIENT ID"])

    return matching_ids

# Apply matching function
df["Matching Patient IDs"] = df.apply(lambda row: find_matching_patients(row, data_for_lift), axis=1)

# Drop helper column
df = df.drop(columns=["Feature Dict"])

In [ ]:
final_df